In [1]:
import numpy as np
import os
import sys
import pickle as pkl
import pandas as pd

In [2]:
sys.path.append('src')
from datasets import dataset_names

In [3]:
data_dir = 'data'

In [4]:
algos = ['tp', 'lr']
algo_names = ['TP-MLC', 'LR-CV']

In [5]:
metrics = ['Precision@3', 'Precision@5', 'Precision@K']

In [6]:
indices = pd.MultiIndex.from_product([dataset_names, algo_names], names=['Dataset', 'Method'])

In [14]:
df_train = pd.DataFrame(index=indices, columns=metrics)
df_test  = pd.DataFrame(index=indices, columns=metrics)

In [15]:
fname = os.path.join(data_dir, 'prec-tp.pkl')
records = pkl.load(open(fname, 'rb'))

In [16]:
records

{'bibtex': {'Test': {'Precision@3': (0.39310801855533467,
    0.0056420225028509163),
   'Precision@5': (0.28827037773359843, 0.004089129789854499),
   'Precision@K': (0.50436113383751091, 0.0077660577802836561)},
  'Train': {'Precision@3': (0.6430327868852459, 0.0041116973534355552),
   'Precision@5': (0.44204918032786883, 0.0036432300240666396),
   'Precision@K': (0.96465342887986327, 0.0020177837505827136)}},
 'scene': {'Test': {'Precision@3': (0.3455964325529543, 0.0030066931859594973),
   'Precision@5': (0.21672240802675585, 0.0016523803574544961),
   'Precision@K': (0.73662207357859533, 0.012444886193761868)},
  'Train': {'Precision@3': (0.34957335535370221, 0.0023698369533023717),
   'Precision@5': (0.2122213047068538, 0.0014156792365537137),
   'Precision@K': (0.88521882741535918, 0.0089718173224213119)}},
 'yeast': {'Test': {'Precision@3': (0.53871319520174477, 0.010565917513821665),
   'Precision@5': (0.45343511450381679, 0.0083494869890093151),
   'Precision@K': (0.509365425

In [18]:
for ds in dataset_names:
    if ds in records:
        for metric in metrics:
            df_train.loc[(ds, 'TP-MLC'), metric] = records[ds]['Train'][metric][0]
            df_test.loc [(ds, 'TP-MLC'), metric] = records[ds]['Test' ][metric][0]

In [28]:
tab_train = df_train.to_latex(float_format=lambda x: '%.4f' % x, na_rep='-', multirow=True)
print(tab_train)

\begin{tabular}{llllll}
\toprule
       &       & Precision@3 & Precision@5 & Precision@K & F1 \\
Dataset & Method &             &             &             &    \\
\midrule
\multirow{2}{*}{yeast} & TP-MLC &      0.6020 &      0.5063 &      0.5690 &  - \\
       & LR-CV &           - &           - &           - &  - \\
\cline{1-6}
\multirow{2}{*}{scene} & TP-MLC &      0.3496 &      0.2122 &      0.8852 &  - \\
       & LR-CV &           - &           - &           - &  - \\
\cline{1-6}
\multirow{2}{*}{bibtex} & TP-MLC &      0.6430 &      0.4420 &      0.9647 &  - \\
       & LR-CV &           - &           - &           - &  - \\
\cline{1-6}
\multirow{2}{*}{bookmarks} & TP-MLC &           - &           - &           - &  - \\
       & LR-CV &           - &           - &           - &  - \\
\cline{1-6}
\multirow{2}{*}{delicious} & TP-MLC &           - &           - &           - &  - \\
       & LR-CV &           - &           - &           - &  - \\
\cline{1-6}
\multirow{2}{*}{mediam

In [ ]:
datasets = ['yeast', 'scene', 'bibtex', 'bookmarks']

In [ ]:
#split = 'Train'
split = 'Test'

In [ ]:
# load perf records
perf_files = [os.path.join(data_dir, 'perf-' + algo + '.pkl') for algo in algos]
perf_dicts = [pkl.load(open(fname, 'rb')) for fname in perf_files]

def perfstr(perftuple):
    return '$%.4f \\pm  %.3f$' % (perftuple[0], perftuple[1])

for metric in metrics:
    tabstr = []
    tabstr.append('\\begin{table}[!h]')
    tabstr.append('\centering')
    tabstr.append('\\caption{Performance in terms of average %s (%s)}' % (metric, split))
    tabstr.append('\\begin{tabular}{lcccc} \\\\ \\hline \\hline')
    tabstr.append(' & ' + ' & '.join([ds for ds in datasets]) + ' \\\\')
    for j in range(len(algos)):
        tabstr.append(algo_names[j] + ' & '
                      + ' & '.join([perfstr(perf_dicts[j][ds][split][metric]) \
                                    if ds in perf_dicts[j] else '-' for ds in datasets ])
                      + ' \\\\')
    tabstr.append('\\hline')
    tabstr.append('\\end{tabular}')
    tabstr.append('\\end{table}')
    print('\n'.join(tabstr))
    print()